In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Description:**

This notebook aims to build a generic interaction checker that leverages Mistral AI and LlamaIndex to analyze potential interactions between drugs and genomic data. The notebook provides a step-by-step guide to set up the environment, process input data, and query the models to get interaction insights, making it a valuable tool for healthcare applications.

This notebook is developed by **Kato Steven Mubiru**

**Steps to Follow**
Setup and Installation: Install the required libraries using the pip commands provided.
API Key Configuration: Replace <YOUR_MISTRALAI_API_KEY> with your actual Mistral AI API key.
Load Data: Since this is a generic implementation, it uses placeholder data for demonstration. Users can replace this with their own data.
Run the Example: Run the example code in a Jupyter notebook or a Python script to see the output.
Enhancements

**This implementation includes:**

Error handling for insufficient drug inputs.
Placeholder data for drug interactions.
Integration of LlamaIndex and Mistral AI.
Functions for querying both LlamaIndex and Mistral AI.
Combined results from both sources.
This generic implementation can be easily adapted to real data and further enhanced with more sophisticated data handling and additional use cases. It provides a solid foundation for contributing to the Mistral AI Cookbook.

In [ ]:
# Install the required libraries
!pip install mistralai llama-index pandas

# Import necessary libraries
import os
import pandas as pd
import mistralai
from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex, MistralAI, MistralAIEmbedding

# Set up Mistral AI API key
os.environ['MISTRAL_API_KEY'] = '<YOUR_MISTRALAI_API_KEY>'

# Setup LLM and Embedding Model
llm = MistralAI(model='mistral-large')
embed_model = MistralAIEmbedding()

# Configure LlamaIndex settings
from llama_index import Settings
Settings.llm = llm
Settings.embed_model = embed_model

# Placeholder: User-provided drug interaction data (for demonstration purposes)
# Users will provide their own data
drug_interactions_data = pd.DataFrame({
    'Drug1': ['Aspirin', 'Ibuprofen', 'Aspirin'],
    'Drug2': ['Warfarin', 'Warfarin', 'Ibuprofen'],
    'Interaction': [
        'Increased risk of bleeding.',
        'Increased risk of bleeding.',
        'Increased risk of gastrointestinal bleeding.'
    ]
})

# Load data into LlamaIndex
documents = SimpleDirectoryReader(input_files=["drug_interactions.csv"]).load_data()
index = GPTVectorStoreIndex.from_documents(documents)

# Define functions for drug and genomics interaction checking
def check_drug_interactions(drugs: List[str]) -> str:
    if len(drugs) < 2:
        return "Please provide at least two drugs to check for interactions."

    # Query LlamaIndex for drug interactions
    query = " and ".join(drugs)
    llama_results = index.query(query)

    # Use Mistral AI API for drug interaction checking
    prompt = f"Check for interactions between the following drugs: {', '.join(drugs)}."
    mistral_results = client.chat([{"role": "user", "content": prompt}])["choices"][0]["message"]["content"]

    # Handle errors and edge cases
    if not llama_results:
        llama_results = "No interactions found in LlamaIndex."

    # Format and return the results
    return f"LlamaIndex Results:\n{llama_results}\n\nMistral AI Results:\n{mistral_results}"

def query_llama_index(drugs: List[str], genes: List[str], variants: List[str]) -> str:
    # Combine queries for drugs, genes, and variants
    drug_query = " and ".join(drugs)
    gene_query = " and ".join(genes)
    variant_query = " and ".join(variants)
    combined_query = f"{drug_query} {gene_query} {variant_query}"
    
    # Query LlamaIndex
    llama_results = index.query(combined_query)
    return llama_results

def query_mistral_ai_api(drugs: List[str], genes: List[str], variants: List[str]) -> str:
    # Create prompt for Mistral AI
    drug_prompt = f"Check for interactions between the following drugs: {', '.join(drugs)}."
    gene_prompt = f"Check for interactions between the following genes: {', '.join(genes)}."
    variant_prompt = f"Check for information on the following genomic variants: {', '.join(variants)}."
    
    # Get responses from Mistral AI
    drug_response = client.chat([{"role": "user", "content": drug_prompt}])["choices"][0]["message"]["content"]
    gene_response = client.chat([{"role": "user", "content": gene_prompt}])["choices"][0]["message"]["content"]
    variant_response = client.chat([{"role": "user", "content": variant_prompt}])["choices"][0]["message"]["content"]
    
    # Combine responses
    combined_results = f"Drug Interactions:\n{drug_response}\n\nGene Interactions:\n{gene_response}\n\nGenomic Variants:\n{variant_response}"
    return combined_results

def check_drug_genomic_interactions(drugs: List[str], genes: List[str], variants: List[str]) -> str:
    # Query LlamaIndex for drug interactions, drug-gene interactions, and genomic variant information
    llama_results = query_llama_index(drugs, genes, variants)

    # Use Mistral AI API for drug interaction checking, drug-gene interactions, and pharmacogenomics recommendations
    mistral_results = query_mistral_ai_api(drugs, genes, variants)

    # Handle errors and edge cases
    if not llama_results:
        llama_results = "No interactions found in LlamaIndex."
    if not mistral_results:
        mistral_results = "No interactions found using Mistral AI."

    # Format and return the results
    return f"LlamaIndex Results:\n{llama_results}\n\nMistral AI Results:\n{mistral_results}"

# Example usage
drugs = ["Aspirin", "Warfarin"]
genes = ["CYP2C9"]
variants = ["rs1799853"]
result = check_drug_genomic_interactions(drugs, genes, variants)
print(result)
